<a href="https://colab.research.google.com/github/rohitpj/Exeter-Placement/blob/main/transformersNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
sys.path.append('/content/drive/MyDrive/Exeter-Placement/BTMF_original.py')
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from distutils.util import strtobool
import sys
import BTMF_original
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
#@title
def convert_tsf_to_dataframe(full_file_path_and_name, replace_missing_vals_with="NaN", value_column_name="series_value",):
    col_names = []
    col_types = []
    all_data = {}
    line_count = 0
    frequency = None
    forecast_horizon = None
    contain_missing_values = None
    contain_equal_length = None
    found_data_tag = False
    found_data_section = False
    started_reading_data_section = False

    with open(full_file_path_and_name, "r", encoding="cp1252") as file:
        for line in file:
            # Strip white space from start/end of line
            line = line.strip()

            if line:
                if line.startswith("@"):  # Read meta-data
                    if not line.startswith("@data"):
                        line_content = line.split(" ")
                        if line.startswith("@attribute"):
                            if (
                                len(line_content) != 3
                            ):  # Attributes have both name and type
                                raise Exception("Invalid meta-data specification.")

                            col_names.append(line_content[1])
                            col_types.append(line_content[2])
                        else:
                            if (
                                len(line_content) != 2
                            ):  # Other meta-data have only values
                                raise Exception("Invalid meta-data specification.")

                            if line.startswith("@frequency"):
                                frequency = line_content[1]
                            elif line.startswith("@horizon"):
                                forecast_horizon = int(line_content[1])
                            elif line.startswith("@missing"):
                                contain_missing_values = bool(
                                    strtobool(line_content[1])
                                )
                            elif line.startswith("@equallength"):
                                contain_equal_length = bool(strtobool(line_content[1]))

                    else:
                        if len(col_names) == 0:
                            raise Exception(
                                "Missing attribute section. Attribute section must come before data."
                            )

                        found_data_tag = True
                elif not line.startswith("#"):
                    if len(col_names) == 0:
                        raise Exception(
                            "Missing attribute section. Attribute section must come before data."
                        )
                    elif not found_data_tag:
                        raise Exception("Missing @data tag.")
                    else:
                        if not started_reading_data_section:
                            started_reading_data_section = True
                            found_data_section = True
                            all_series = []

                            for col in col_names:
                                all_data[col] = []

                        full_info = line.split(":")

                        if len(full_info) != (len(col_names) + 1):
                            raise Exception("Missing attributes/values in series.")

                        series = full_info[len(full_info) - 1]
                        series = series.split(",")

                        if len(series) == 0:
                            raise Exception(
                                "A given series should contains a set of comma separated numeric values. At least one numeric value should be there in a series. Missing values should be indicated with ? symbol"
                            )

                        numeric_series = []

                        for val in series:
                            if val == "?":
                                numeric_series.append(replace_missing_vals_with)
                            else:
                                numeric_series.append(float(val))

                        if numeric_series.count(replace_missing_vals_with) == len(
                            numeric_series
                        ):
                            raise Exception(
                                "All series values are missing. A given series should contains a set of comma separated numeric values. At least one numeric value should be there in a series."
                            )

                        all_series.append(pd.Series(numeric_series).array)

                        for i in range(len(col_names)):
                            att_val = None
                            if col_types[i] == "numeric":
                                att_val = int(full_info[i])
                            elif col_types[i] == "string":
                                att_val = str(full_info[i])
                            elif col_types[i] == "date":
                                att_val = datetime.strptime(
                                    full_info[i], "%Y-%m-%d %H-%M-%S"
                                )
                            else:
                                raise Exception(
                                    "Invalid attribute type."
                                )  # Currently, the code supports only numeric, string and date types. Extend this as required.

                            if att_val is None:
                                raise Exception("Invalid attribute value.")
                            else:
                                all_data[col_names[i]].append(att_val)

                line_count = line_count + 1

        if line_count == 0:
            raise Exception("Empty file.")
        if len(col_names) == 0:
            raise Exception("Missing attribute section.")
        if not found_data_section:
            raise Exception("Missing series information under data section.")

        all_data[value_column_name] = all_series
        loaded_data = pd.DataFrame(all_data)

        return (
            loaded_data,
            frequency,
            forecast_horizon,
            contain_missing_values,
            contain_equal_length,
        )
# Example of usage
# loaded_data, frequency, forecast_horizon, contain_missing_values, contain_equal_length = convert_tsf_to_dataframe("TSForecasting/tsf_data/sample.tsf")


In [3]:
def trim_dataframe(filename,columnname):
    loaded_data, frequency, forecast_horizon, contain_missing_values, contain_equal_length = convert_tsf_to_dataframe(filename)
    building_data = loaded_data[loaded_data['series_name'].str.contains(columnname)]
    max_start_timestamp = building_data['start_timestamp'].max()
    building_data['num_timestamps'] = building_data['series_value'].apply(len)
    min_timestamps = building_data['num_timestamps'].min()
    building_data['uniform_series'] = building_data['series_value'].apply(lambda x: x[-min_timestamps:])
    building_data['num_timestamps'] = building_data['uniform_series'].apply(len)
    return building_data['uniform_series']


In [4]:
def impute_dataframe(dataframe, rank, time_lags, burn_iter, gibbs_iter, option = "factor"):
    dense_tensor = dataframe
    dim = dense_tensor.shape
    list_of_arrays = [np.array(series) for series in dense_tensor]
    # Stack these arrays vertically to form a 2D matrix
    dense_mat_2d = np.vstack(list_of_arrays)
    sparse_mat = dense_mat_2d.copy()
    dense_mat_2d = np.where(dense_mat_2d == 'NaN', np.nan, dense_mat_2d).astype(float)
    sparse_mat = np.where(sparse_mat == 'NaN', np.nan, sparse_mat).astype(float)
    del dense_tensor
    dim1, dim2 = sparse_mat.shape
    init = {"W": 0.1 * np.random.randn(dim1, rank), "X": 0.1 * np.random.randn(dim2, rank)}
    mat, W, X, A= BTMF_original.BTMF(dense_mat_2d, sparse_mat, init, rank, time_lags, burn_iter, gibbs_iter)
    # Assuming you have column names and indices stored
    df=pd.DataFrame(mat)
    return df

In [5]:
NaN_df=trim_dataframe("/content/drive/MyDrive/Exeter-Placement/Challenge/phase_1 data/phase_1_data/phase_1_data.tsf","Building")
time_lags = np.array([1, 4, 96])
burn_iter=0
gibbs_iter=1
rank=10

df=impute_dataframe(NaN_df,rank,time_lags,burn_iter,gibbs_iter)
scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df.T).T, columns=df.columns, index=df.index)

sequence_length = 8640  # 3 months of data
prediction_length = 2880  # 1 month of data

train_end = 41572 - prediction_length - sequence_length
train_data = df_scaled.iloc[:, :train_end]
validation_data = df_scaled.iloc[:, train_end:train_end+sequence_length]
test_data = df_scaled.iloc[:, train_end+sequence_length:train_end+2*sequence_length]


<ipython-input-3-2cfb269e37d8>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  building_data['num_timestamps'] = building_data['series_value'].apply(len)
<ipython-input-3-2cfb269e37d8>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  building_data['uniform_series'] = building_data['series_value'].apply(lambda x: x[-min_timestamps:])
<ipython-input-3-2cfb269e37d8>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

BTMF Iteration: 0


In [6]:
def create_sequences(data, seq_length, pred_length):
    sequences = []
    target_sequences = []

    for i in range(len(data.columns) - seq_length - pred_length + 1):
        sequences.append(data.iloc[:, i:i+seq_length].values)
        target_sequences.append(data.iloc[:, i+seq_length:i+seq_length+pred_length].values)

    return np.array(sequences), np.array(target_sequences)

In [7]:
X_train, Y_train = create_sequences(train_data, sequence_length, prediction_length)
X_val, Y_val = create_sequences(validation_data, sequence_length, prediction_length)


In [8]:
def positional_encoding(seq_length, d_model):
    position = np.arange(seq_length)[:, np.newaxis]
    div_term = np.exp(np.arange(0, d_model, 2) * -(np.log(10000.0) / d_model))
    pos_enc = np.zeros((d_model, seq_length))
    pos_enc[0::2, :] = np.sin(position * div_term).T
    pos_enc[1::2, :] = np.cos(position * div_term).T
    return pos_enc[np.newaxis, :, :]


In [9]:
d_model = 6  # Number of sensors
# Generate positional encoding
pos_enc = positional_encoding(sequence_length, d_model)
print(pos_enc.shape, type(pos_enc))
# Add positional encoding to X_train
X_val = validation_data.values[np.newaxis, :, :]
X_train += pos_enc
X_val += pos_enc


(1, 6, 8640) <class 'numpy.ndarray'>


In [10]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        assert d_model % self.num_heads == 0
        self.depth = d_model // self.num_heads

        self.wq = nn.Linear(8640, d_model)
        self.wk = nn.Linear(8640, d_model)
        self.wv = nn.Linear(8640, d_model)

        self.dense = nn.Linear(d_model, d_model)

    def split_heads(self, x, batch_size):
        x = x.view(batch_size, -1, self.num_heads, self.depth)
        return x.permute(0, 2, 1, 3)

    def forward(self, v, k, q, mask=None):
        batch_size = q.size(0)

        q = self.split_heads(self.wq(q), batch_size)
        k = self.split_heads(self.wk(k), batch_size)
        v = self.split_heads(self.wv(v), batch_size)

        matmul_qk = torch.matmul(q, k.permute(0, 1, 3, 2))
        d_k = self.depth ** 0.5
        scaled_attention_logits = matmul_qk / d_k

        if mask is not None:
            scaled_attention_logits += (mask * -1e9)

        attention_weights = F.softmax(scaled_attention_logits, dim=-1)
        output = torch.matmul(attention_weights, v)

        output = output.permute(0, 2, 1, 3).contiguous().view(batch_size, -1, self.d_model)
        return self.dense(output), attention_weights

In [11]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, dff, dropout_rate=0.1):
        super(EncoderLayer, self).__init__()

        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = nn.Sequential(
          nn.Linear(d_model, dff),
          nn.ReLU(),
          nn.Linear(dff, d_model)
        )


        self.layernorm1 = nn.LayerNorm(d_model)
        self.layernorm2 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout_rate)
        self.dropout2 = nn.Dropout(dropout_rate)

    def forward(self, x, training, mask=None):
        print("start")
        attn_output, _ = self.mha(x, x, x, mask)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)

        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)

        return out2


In [12]:
"""
class Encoder(nn.Module):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, max_position_encoding, dropout_rate=0.1):
        super(Encoder, self).__init__()
        print("dnfhgafg")
        self.d_model = d_model
        self.num_layers = num_layers
        self.pos_encoding = torch.tensor(positional_encoding(max_position_encoding, d_model), dtype=torch.float32)
        self.enc_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, dff, dropout_rate) for _ in range(num_layers)])
        self.dropout = nn.Dropout(dropout_rate)
        print(type(self.pos_encoding))  # Check the type of self.pos_encoding
        print(self.pos_encoding.device)  # Check the device of self.pos_encoding if it's a tensor
        print("yo")
    def forward(self, x, training, mask=None):
        print("heloooooo")
        seq_len = x.size(1)
        self.pos_encoding = self.pos_encoding.clone().detach().to(x.device)
        x *= torch.sqrt(torch.tensor(self.d_model, dtype=torch.float32))
        #x += self.pos_encoding[:, :seq_len].to(x.device)
        #x += self.pos_encoding[:, :seq_len, :x.size(2)]
        print("x shape",x.shape)
        print("pos encoding shape",self.pos_encoding[:, :seq_len, :].shape)
        x += self.pos_encoding[:, :seq_len, :]
        print(x.shape)
        print(self.pos_encoding[:, :seq_len, :x.size(2)].shape)
        x = self.dropout(x)
        for i in range(self.num_layers):
            x = self.enc_layers[i](x, training, mask)
        return x
"""
class Encoder(nn.Module):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, max_position_encoding, dropout_rate=0.1):
        super(Encoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        # Create a positional encoding tensor with a sequence length of 8640
        self.pos_encoding = torch.tensor(positional_encoding(6, d_model), dtype=torch.float32)
        #self.pos_encoding = torch.tensor(positional_encoding(8640, d_model), dtype=torch.float32)

        self.enc_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, dff, dropout_rate) for _ in range(num_layers)])
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x, training, mask=None):
        x = x.view(batch_size, -1)
        seq_len = x.size(1)
        self.pos_encoding = self.pos_encoding.clone().detach().to(x.device)
        x *= torch.sqrt(torch.tensor(self.d_model, dtype=torch.float32))

        # Slice the positional encoding tensor to match the sequence length of x
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x)

        for i in range(self.num_layers):
            x = self.enc_layers[i](x, training, mask)

        return x


In [13]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, dff, dropout_rate=0.1):
        super(DecoderLayer, self).__init__()

        self.mha1 = MultiHeadAttention(d_model, num_heads)
        self.mha2 = MultiHeadAttention(d_model, num_heads)

        self.ffn = nn.Sequential(
            nn.Linear(d_model, dff),
            nn.ReLU(),
            nn.Linear(dff, d_model)
        )

        self.layernorm1 = nn.LayerNorm(d_model)
        self.layernorm2 = nn.LayerNorm(d_model)
        self.layernorm3 = nn.LayerNorm(d_model)

        self.dropout1 = nn.Dropout(dropout_rate)
        self.dropout2 = nn.Dropout(dropout_rate)
        self.dropout3 = nn.Dropout(dropout_rate)

    def forward(self, x, enc_output, training, look_ahead_mask=None, padding_mask=None):
        attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)
        attn1 = self.dropout1(attn1, training=training)
        out1 = self.layernorm1(attn1 + x)

        attn2, attn_weights_block2 = self.mha2(enc_output, enc_output, out1, padding_mask)
        attn2 = self.dropout2(attn2, training=training)
        out2 = self.layernorm2(attn2 + out1)

        ffn_output = self.ffn(out2)
        ffn_output = self.dropout3(ffn_output, training=training)
        out3 = self.layernorm3(ffn_output + out2)

        return out3, attn_weights_block1, attn_weights_block2


In [14]:
class Decoder(nn.Module):
    def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size, max_position_encoding, dropout_rate=0.1):
        super(Decoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.pos_encoding = torch.tensor(positional_encoding(max_position_encoding, d_model), dtype=torch.float32)


        self.dec_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, dff, dropout_rate) for _ in range(num_layers)])
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x, enc_output, training, look_ahead_mask=None, padding_mask=None):
        seq_len = x.size(1)
        attention_weights = {}

        # Remove the embedding line
        # x = self.embedding(x) * torch.sqrt(torch.tensor(self.d_model))

        x *= torch.sqrt(torch.tensor(self.d_model, dtype=torch.float32))
        #x += self.pos_encoding[:, :seq_len].to(x.device)
        x += self.pos_encoding[:, :seq_len, :x.size(2)]
        x = self.dropout(x)

        for i in range(self.num_layers):
            x, block1, block2 = self.dec_layers[i](x, enc_output, training, look_ahead_mask, padding_mask)
            attention_weights[f'decoder_layer{i+1}_block1'] = block1
            attention_weights[f'decoder_layer{i+1}_block2'] = block2

        return x, attention_weights



In [15]:
class Transformer(nn.Module):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, target_vocab_size, pe_input, pe_target, dropout_rate=0.1):
        super(Transformer, self).__init__()

        self.encoder = Encoder(num_layers, d_model, num_heads, dff, input_vocab_size, pe_input, dropout_rate)
        self.decoder = Decoder(num_layers, d_model, num_heads, dff, target_vocab_size, pe_target, dropout_rate)

        self.final_layer = nn.Linear(d_model, target_vocab_size)

    def forward(self, inp, tar, training, enc_padding_mask=None, look_ahead_mask=None, dec_padding_mask=None):
        enc_output = self.encoder(inp, training, enc_padding_mask)
        dec_output, attention_weights = self.decoder(tar, enc_output, training, look_ahead_mask, dec_padding_mask)

        final_output = self.final_layer(dec_output)
        return final_output, attention_weights


In [16]:
import torch.optim as optim

input_vocab_size = 512
target_vocab_size = 512
max_position_encoding_input = 9000  # A bit more than 8640 to ensure coverage
max_position_encoding_target = 9000

# Create the Transformer model instance
model = Transformer(num_layers=6, d_model=512, num_heads=8, dff=2048,
                    input_vocab_size=input_vocab_size, target_vocab_size=target_vocab_size,
                    pe_input=max_position_encoding_input, pe_target=max_position_encoding_target)

# Now, you can define your optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [17]:
import torch.optim as optim

criterion = nn.MSELoss()  # Mean Squared Error Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer with a learning rate of 0.001


In [18]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
class TimeSeriesDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

train_dataset = TimeSeriesDataset(X_train, Y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)


In [ ]:
model = Transformer(num_layers=6, d_model=512, num_heads=8, dff=2048,
                    input_vocab_size=512, target_vocab_size=512,
                    pe_input=9000, pe_target=9000)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
num_epochs = 1

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        data = data.float()
        print("Input data shape:", data.shape)
        target = target.float()

        # Forward pass
        # Exclude the last time step from target as input to the decoder
        outputs, _ = model(data, target[:, :-1], training=True)

        # Compute the loss
        # Exclude the first time step from target to match the shape of outputs
        loss = criterion(outputs, target[:, 1:])

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss / len(train_loader):.4f}")

